<center>

# EE2703 - Assignment 8 (Code)

## Name : ANIRUDH B S
## Roll No : EE21B019
</center>

In [1]:
%load_ext Cython

The above cell loads the cython extension

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

The above cell is inserted so that the entire annotated cython code appears as a continuous chunk of code as output below the below cell.

In [3]:
%%cython --annotate 
import sys
import numpy as np
cimport numpy as np
import cmath
import cython

@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.cdivision(True)
cpdef GaussianElimination(np.complex128_t [:,:] A,np.complex128_t [:] b): #Solves a system of linear equations given matrices A and b using Gaussian Elimination with parital pivoting (swapping of rows included)
    cdef int n,m, i, j,k 
    n = A.shape[0]
    m = A.shape[1]
    if m!=n :
        sys.exit("Inconsistent Equation")
    cdef np.complex128_t [:,:] aug = np.array([[0 for j in range(n+1)] for i in range(n)],dtype = complex)
    cdef np.complex128_t [:] x = np.array([0 for i in range(n)], dtype = complex)  #Solution is initialised to zeros
    cdef complex r 
    aug[:,:-1] = A
    aug[:,-1] = b # Create the augumented matrix [A|b] by concatenating A and b column wise
    try :
        for i in range(n-1):
            if aug[i][i] == 0: #Checks if pivot is zero
                for l in range(i+1, n):
                    if abs(aug[l][i]) > abs(aug[i][i]):
                        aug[i], aug[l] = aug[l], aug[i] #Swaps two rows of the augmented matrix
            for j in range (i+1,n):
                if aug[i][i] == 0 :
                    raise ZeroDivisionError("Singular Matrix") #Raises exception
                else :
                    r = aug[j][i]/aug[i][i] #Find the ratio for each row
                for k in range(i,n+1):
                    aug[j][k] = aug[j][k] - r*aug[i][k] #Convert matrix to Row Echelon Matrix
        if aug[n-1][n-1] == 0:
            raise ZeroDivisionError("Singular Matrix") #Raises exception
        else :
            x[n-1] = aug[n-1][n]/aug[n-1][n-1] #Start of Back Substitution
        for i in range(n-2,-1,-1):
            x[i] = aug[i][n] 
            for j in range(i+1,n):
                x[i] = x[i] - aug[i][j]*x[j] #Back Substitution
            try :
                x[i] = x[i]/aug[i][i]
            except :
                print("Singular Matrix")
        return np.array(x, dtype = complex)
    except ZeroDivisionError:
        print("Singular Matrix")
        
cdef int n = 0            #Number of nodes in circuit
cdef int k = 0            #Number of independent voltage sources in circuit
cdef list r = []          #Stores list of resistance objects
cdef list v = []          #Stores list of voltage sources objects
cdef list cur = []        #Stores list of current source objects
cdef list c = []          #Stores list of capacitor objects
cdef list ll = []         #Stores list of inductor objects
cdef dict nodes = {}      #Stores dictionary mapping of nodes
cdef set w = set()        #Stores set of distinct frequencies
cdef float frequency  = 0 #Stores last entered frequency of source
cdef float phase = 0      #Stores phase of complex quantity
    
cdef class R:             #Created a class for storing R objects nicely
    cdef float value 
    cdef (int, int) nodes 
    def __init__(self, int n1, int n2, float value) :
        self.value = value
        self.nodes = (n1, n2)
    cpdef (int, int) getnodes(self):
        return self.nodes
    cpdef float getvalue(self):
        return self.value

cdef class L:            #Created a class for storing L objects nicely
    cdef float value 
    cdef (int, int) nodes 
    def __init__(self, int n1, int n2, float value) :
        self.value = value
        self.nodes = (n1, n2)
    cpdef (int, int) getnodes(self):
        return self.nodes
    cpdef float getvalue(self):
        return self.value
        
cdef class C:            #Created a class for storing C objects nicely
    cdef float value 
    cdef (int, int) nodes 
    def __init__(self, int n1, int n2, float value) :
        self.value = value
        self.nodes = (n1, n2)
    cpdef (int, int) getnodes(self):
        return self.nodes
    cpdef float getvalue(self):
        return self.value

cdef class V:            #Created a class for storing V objects nicely 
    cdef complex value 
    cdef (int, int) nodes 
    def __init__(self, int n1, int n2,complex value) :
        self.value = value
        self.nodes = (n1, n2)
    cpdef (int, int) getnodes(self):
        return self.nodes
    cpdef complex getvalue(self):
        return self.value

cdef class I:            #Created a class for storing I objects nicely
    cdef complex value 
    cdef (int, int) nodes 
    def __init__(self, int n1, int n2, complex value) :
        self.value = value
        self.nodes = (n1, n2)
    cpdef (int, int) getnodes(self):
        return self.nodes
    cpdef complex getvalue(self):
        return self.value
        
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)        
cdef int findckt(list lines) : #Created a function to find .circuit in netlist
    cdef str line
    cdef int n 
    cdef int i
    n = len(lines)
    for i in range(n) : 
        line = lines[i]
        if line == ".circuit" or line == ".circuit\n" :
            return i
    return -1

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False) 
cdef int findend(list lines) : #Created a function to find .end in netlist
    cdef str line  
    cdef int n 
    cdef int i
    n = len(lines)
    for i in range(n) : 
        line = lines[i]
        if line == ".end" or line == ".end\n" :
            return i
    return -1


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False) 
@cython.cdivision(True)
def getMR (M, tuple nodes, float value) :               #Given a resistor updates matrix M
    cdef int n1, n2 
    n1, n2 = nodes
    M[n1][n1] += 1/value 
    M[n1][n2] -= 1/value
    M[n2][n1] -= 1/value
    M[n2][n2] += 1/value
    return M
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False) 
@cython.cdivision(True)
def getMC(tuple nodes, M, float frequency, float value):#Given a capacitor updates matrix M
    cdef int n1, n2
    n1, n2 = nodes
    M[n1][n1] += complex(0,2*np.pi*frequency*value)
    M[n1][n2] -= complex(0,2*np.pi*frequency*value)
    M[n2][n1] -= complex(0,2*np.pi*frequency*value)
    M[n2][n2] += complex(0,2*np.pi*frequency*value)
    return M
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False) 
@cython.cdivision(False)
def getML(tuple nodes, M, b,float frequency, float value, int idx): #Given a inductor updates the matrix M
    cdef int n1, n2
    n1, n2 = nodes
    global n, k
    try :
        M[n1][n1] += complex(0,1/(6.28*frequency*value))
        M[n1][n2] -= complex(0,1/(6.28*frequency*value))
        M[n2][n1] -= complex(0,1/(6.28*frequency*value))
        M[n2][n2] += complex(0,1/(6.28*frequency*value))
    except ZeroDivisionError :
        M[n1][n+k+idx] += 1
        M[n+k+idx][n2] -= 1
        M[n+k+idx][n1] += 1
        M[n2][n+k+idx] -= 1
        b[n+k+idx] = 0
    return M, b

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False) 
@cython.cdivision(True)     
def getMV (M, b, tuple nodes, complex value, int n, int idx):#Created a function to change M after considering voltage sources
    cdef int n1, n2
    n1, n2 = nodes
    M[n1][n+idx] += 1
    M[n+idx][n1] += 1
    M[n+idx][n2] -= 1
    M[n2][n+idx] -= 1
    b[n+idx] = value
    return M, b
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False) 
@cython.cdivision(True)   
def getMI (b, tuple nodes, complex value) : #Created a function to change M after considering current sources
    cdef int n1, n2
    n1, n2 = nodes
    b[n1] += value
    b[n2] -= value
    return b

cdef reset() : #Function to reset after all the operations with respect to a netlist are completed.
    global r
    global v
    global cur
    global c
    global ll
    global n
    global k
    global w
    global frequency
    global phase
    #I have imported all the global variables used and reset them all to be zero
    nodes = {}
    r = []
    v = []
    cur = []
    c = []
    ll = []
    n = 0
    k = 0
    w.clear() 
    frequency = 0
    phase = 0
    
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False) 
@cython.cdivision(True)   
cpdef main(str name):
    reset()
    cdef list lines
    if name[-8:] != ".netlist" :       #Check for .netlist in ending
        print(".netlist not found")
        sys.exit("Netlist should end with .netlist")
    with open(name) as f:
        lines = f.readlines()
    cdef int cktcount                 #Used to check for number of .circuit in netlist
    cdef int endcount                 #Used to check for number of .end in netlist
    cdef str line                     #Used to store line of netlist       
    cdef list s                       #Store words of a line of netlist
    cdef int a                        #Checks whether .circuit is present or not
    cdef int B                        #Checks whether .end is present or not
    cktcount = 0
    endcount = 0
    cdef int lv                       #Looping variable
    cdef int lenlines                 #Variable to store length og lines
    global w                         #Use the set of all W (frequencies)
    lenlines = len(lines)
    for lv in range(lenlines) :
        line = lines[lv]
        s = []
        if line == "" or line == "\n":
            continue
        s = line.split()
        if s[0] == ".ac" :           #If ac frequency is found, add it to w    
            w.add(float(s[2]))
        if line == ".circuit" or line == ".circuit\n":
            cktcount +=1
        if line == ".end" or line == ".end\n":
            endcount +=1
    if cktcount > 1 : #Check for multiple .circuit
        print("Too many .circuit s")
        sys.exit("Netlist should contain only one .circuit")
    if endcount > 1 : #Check for multiple .end
        print("Too many .end s")
        sys.exit("Netlist should contain only one .end")
    a = findckt(lines)
    B = findend(lines)
    if a == -1 :
        print (".circuit not found")
    if B == -1 :
        print (".end not found")
    global nodes
    nodes = {} #Stores list of nodes in the circuit and their mapping
    cdef int count
    count = 1
    global n
    global k
    global r 
    global c
    global ll
    global v 
    global cur
    global frequency
    global phase
    #Using global keyword we access the lists and variables that are used throughout the code
    cdef list circuit       #Stores the actual content of the netlist
    cdef list ele           #Parses each individual element of a line
    cdef int k_             #Runs through 1 and 2, that is the index of nodes given a line in the netlist
    cdef str l              #Stores a line of circuit
    cdef int lenckt         #Stores length of circuit 
    cdef int i              #Looping variable
    if a != -1 and B!=-1 :
        circuit = lines[a+1:B]
        lenckt = len(circuit)
        for i in range(lenckt) :
            l = circuit[i]
            ele = l.split()
            for k_ in range(1,3) :
                if ele[k_] == "GND" :      #GND is assigned a voltage of zero
                    nodes[ele[k_]] = 0
                if ele[k_] not in nodes.keys():
                    nodes[ele[k_]] = count
                    count += 1
            if l[0] == 'R': #Storing a resistor nicely
                print("Found a resistor")
                ro = R(nodes[ele[1]], nodes[ele[2]],  float(ele[3]))
                r.append(ro)
            if l[0] == 'C': #Storing a capacitor nicely
                print("Found a capacitor")
                co = C(nodes[ele[1]], nodes[ele[2]], float(ele[3]))
                c.append(co)
            if l[0] == 'L': #Storing a inductor nicely
                print("Found a inductor")
                lo = L(nodes[ele[1]], nodes[ele[2]], float(ele[3]))
                ll.append(lo)
            if l[0] == 'V': #Storing a voltage source nicely
                if ele[3] == "ac" : #AC Source
                    phase = float(ele[5])
                    value = complex(float(ele[4])*np.cos(phase), float(ele[4])*np.sin(phase))
                elif ele[3] == "dc" : #DC source
                    value = float(ele[4])
                    w.add(0)
                else : #Checks if AC/ DC entry is left empty
                    sys.exit("Enter AC or DC")
                vo = V(nodes[ele[1]], nodes[ele[2]],value)
                v.append(vo)
                print("Found a voltage source with value: ", vo.value)
                k = k + 1
            if l[0] == 'I': #Storing a current source nicely
                if ele[3] == "ac" : #AC source
                    phase = float(ele[5])
                    value = complex(float(ele[4])*np.cos(phase), float(ele[4])*np.sin(phase))
                elif ele[3] == "dc" : #DC source
                    value = float(ele[4])
                    w.add(0)
                else : #Checks if AC/ DC entry is left empty
                    sys.exit("Enter AC or DC")
                io = I(nodes[ele[1]], nodes[ele[2]], value)
                cur.append(io)
                print("Found a current source with value: ", io.value)
        n = len(nodes)
        cktsolver(n, k, r, c, ll, v, cur)
        reset()
        
@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False) 
@cython.cdivision(True)
cpdef cktsolver(n, k, r, c, ll, v, cur): #Solves the circuit modified nodal analysis Mx = b
    global w
    cdef int nlist           #Stores number of elements of a particular type at one given time
    cdef int i               #Looping variable
    cdef int lenw = len(w)   #Number of distinct frequencies in netlist
    cdef int gnd = 0         #To check for GND specification
    cdef list nk             #List of nodes keys
    cdef int node_idx        #Looping variable
    cdef int node_values     #List of nodes values
    cdef int lenv            #Length of node_values
    global frequency  
    cdef np.ndarray[np.complex128_t, ndim=2] M #Create matrix M 
    cdef np.ndarray[np.complex128_t, ndim=1] b #Create matrix b
    cdef np.ndarray[np.complex128_t, ndim = 1] x
    if lenw == 1:
        frequency = next(iter(w))
        if frequency == 0:
            M = np.zeros((n+k+len(ll), n+k+len(ll)), dtype = np.complex128)  #M = zero matrix
            b = np.zeros(n+k+len(ll), dtype = np.complex128)     #b = zero vector
            x = np.zeros(n+k+len(ll), dtype = np.complex128) 
        else :
            M = np.zeros((n+k, n+k), dtype = np.complex128)  #M = zero matrix
            b = np.zeros(n+k, dtype = np.complex128)     #b = zero vector
            x = np.zeros(n+k, dtype = np.complex128)
        nlist = len(r)
        for i in range(nlist):
            res = r[i]
            M = getMR(M, res.getnodes(), res.getvalue())                       #Create M for R elements
        nlist = len(c)
        for i in range(nlist):
            cap = c[i]
            M = getMC(cap.getnodes(), M, frequency, cap.getvalue())            #Create M for C elements
        nlist = len(ll)
        for i in range(nlist):
            ind = ll[i]
            M, b = getML(ind.getnodes(), M, b, frequency, ind.getvalue(), i)  #Create M, b for L elements
        nlist = len(v)
        for i in range(nlist):
            vol = v[i]
            M, b = getMV(M, b, vol.getnodes(), vol.getvalue(), i, n)          #Create M, b for V elements
        nlist = len(cur) 
        for i in range(nlist) : 
            curr = cur[i]
            b = getMI(b, curr.getnodes(), curr.getvalue())                    #Create b for I elements
        M[0] = 0 #Set row corresponding to GND to 0
        M[0][0] = 1 #Set the matrix entry corresponding to GND to 1
        b[0] = 0 
        for _ in nodes.keys():
            if _ == 'GND' :
                gnd = 1
        if gnd != 1:
            sys.exit("No GND specified") 
        try :
            x = np.linalg.solve(M,b)     #Solves the set of equations generated
        except :
            sys.exit("No solution")         #Throws error that circuit is invalid
        nk = list(nodes.keys())
        node_values = len(nodes.values())
        lenv = len(v)
        if frequency == 0 :
            for node_idx in range(node_values) :
                print(f"The voltage at node {nk[node_idx]} is {x[node_idx].real}")
            for i in range(lenv):
                vol = v[i]
                print(f"The current through voltage source {vol.getvalue()} V between nodes {vol.getnodes()} is {x[n+v.index(vol)].real}")
        else :
            for node_idx in range(node_values) :
                print(f"The voltage at node {nk[node_idx]} is {abs(x[node_idx])} at an angle {cmath.phase(x[node_idx])} with source.")
            for i in range(lenv):
                vol = v[i]
                print(f"The current through voltage source {vol.getvalue()} V between nodes {vol.getnodes()} is {abs(x[n+v.index(vol)])} at an angle {cmath.phase(x[n+v.index(vol)])} with source.")
    else :
        print("Enter a single frequency ONLY.")

The documentation with explanation has been provided in a separate notebook. This has been done because Cython expects all the code to be in the same cell, however, it will not be a neat documentation. Thus, I have documented the same on another Jupyter notebook converted it into LaTeX and am submitting both the notebooks with the Documentation PDF. 

In [4]:
filename = input("Enter the name of the netlist including extension (.netlist)")

Enter the name of the netlist including extension (.netlist)ckt1.netlist


The above cell takes filename input

In [5]:
main(filename)

Found a resistor
Found a resistor
Found a resistor
Found a resistor
Found a resistor
Found a voltage source with value:  (5+0j)
The voltage at node GND is 0.0
The voltage at node 1 is 0.0
The voltage at node 2 is 0.0
The voltage at node 3 is 0.0
The voltage at node 4 is -5.0
The current through voltage source (5+0j) V between nodes (0, 4) is -0.0005


The above cell runs the Cython code to produce the output

In [ ]:
%timeit main(filename) #Run this after commenting out the print statements - You would'nt like seeing the same result being printed over and over again

The above cell times the Cython code that produces the output